In [4]:
import QuantLib as ql

def european_binomial(S, K, T, r, q, sigma, option_type="call", steps=2000):
  """
  Compute European option price using QuantLib's Binomial model.

  Parameters:
  - S: Stock price
  - K: Strike price
  - T: Time to maturity
  - r: Risk-free rate
  - q: Dividend yield
  - sigma: Volatility
  - option_type: "call" or "put"
  - steps: Number of binomial steps

  Returns:
  - Option price
  """

  # Define the option's data
  day_count = ql.Actual365Fixed()
  today = ql.Date.todaysDate()
  expiry_date = today + ql.Period(f"{int(T*365)}d")

  ql.Settings.instance().evaluationDate = today

  # Convert to QuantLib format

  spot = ql.SimpleQuote(S)

  rate = ql.SimpleQuote(r)
  dividend = ql.SimpleQuote(q)
  volatility = ql.SimpleQuote(sigma)

  # Interest-rate term structure
  flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(rate), day_count))

  # Dividend term structure
  dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(dividend), day_count))

  # Volatility term structure
  flat_vol = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.TARGET(), ql.QuoteHandle(volatility), day_count))

  # Set up the option
  exercise = ql.EuropeanExercise(expiry_date)
  payoff = ql.PlainVanillaPayoff(ql.Option.Call if option_type == "call" else ql.Option.Put, K)

  european_option = ql.VanillaOption(payoff, exercise)

  # Set up the binomial model
  bsm_process = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot), dividend_yield, flat_ts, flat_vol)

  # Set up pricing engine
  binomial_engine = ql.BinomialVanillaEngine(bsm_process, "crr", steps)
  european_option.setPricingEngine(binomial_engine)

  return european_option.NPV()

In [5]:
def american_binomial(S, K, T, r, q, sigma, option_type="call", steps=2000):
  """
  Compute American option price using QuantLib's Binomial model.

  Parameters:
  - S: Stock price
  - K: Strike price
  - T: Time to maturity
  - r: Risk-free rate
  - q: Dividend yield
  - sigma: Volatility
  - option_type: "call" or "put"
  - steps: Number of binomial steps

  Returns:
  - Option price
  """
  # Define the option's data
  day_count = ql.Actual365Fixed()
  today = ql.Date.todaysDate()
  expiry_date =  today + ql.Period(f"{int(T*365)}d")

  ql.Settings.instance().evaluationDate = today

  # Convert to QuantLib format

  spot = ql.SimpleQuote(S)

  rate = ql.SimpleQuote(r)
  dividend = ql.SimpleQuote(q)
  vol = ql.SimpleQuote(sigma)

  # Interest-rate term structure
  flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(rate), day_count))

  # Dividend term structure
  dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(dividend), day_count))

  # Volatility term structure
  flat_vol = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.TARGET(), ql.QuoteHandle(vol), day_count))

  # Set up the option excercise and payoff
  settlement = today
  exercise = ql.AmericanExercise(settlement, expiry_date)
  payoff = ql.PlainVanillaPayoff(ql.Option.Call if option_type == "call" else ql.Option.Put, K)

  american_option = ql.VanillaOption(payoff, exercise)

  # Set up the binomial model
  bsm_process = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot), dividend_yield, flat_ts, flat_vol)

  # Set up pricing engine
  binomial_engine = ql.BinomialVanillaEngine(bsm_process, "crr", steps)
  american_option.setPricingEngine(binomial_engine)

  return american_option.NPV()

In [6]:
import scipy.stats as stat
import pandas as pd

def simulation(N, OPTtype = "call"):
  # Maintain number of simulations within bounds
  if (N <= 100 or N >= 1000000):
    raise Exception("N must be between 100 and 1000000")

  data = []

  for _ in range(N):
    S = stat.uniform.rvs(loc = 80, scale = (120-80)) # Spot price is constant at 100
    # Simulation ranging from minumum = loc and maximum = first term of scale
    K = stat.uniform.rvs(loc = 90, scale = (110-90)) # loc establishes the floor and scale the range size
    q = stat.uniform.rvs(loc = .01, scale = (.03-.01))
    r = stat.uniform.rvs(loc = .01, scale = (.06-.01))
    v = stat.uniform.rvs(loc = .05, scale = (.50-.05))
    t = stat.uniform.rvs(loc = .1, scale = (1-.1))

    european_option_price = european_binomial(S, K, t, r, q, v, option_type = OPTtype) # Use euro function to compute the european option price

    american_option_price = american_binomial(S, K, t, r, q, v, option_type = OPTtype) # Use american function to compute the american option price

    d = {"S" : S, "K" : K, "q" : q, "r" : r, "v" : v, "t" : t,
         "European Option Price" : european_option_price, "American Option Price": american_option_price}
    data.append(d)

  simulation = pd.DataFrame(data)
  return simulation

In [7]:
df = simulation(100000)

In [8]:
from datetime import date
df.to_csv(f"simulated_data_{date.today()}.csv")